In [1]:
import gtfs_kit as gk
import os.path
import geopandas as gpd
import geoplot as gplt
import datetime as dt
import gtfs_kit_scissors

In [3]:
gtfs_dir = "/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/gtfs"
years = [2020, 2021]
paths = {y: os.path.join(gtfs_dir,str(y),str(y)+".zip") for y in years}
#feeds = {y: gk.read_feed(paths[y], dist_units='km') for y in years}

Load feeds for March.

In [4]:
# f20_mar = gtfs_kit_scissors.cut_dates(gk.read_feed(paths[2020], dist_units='km'), ["20200301","20200331"], validate=False)
f20_mar = gk.read_feed(paths[2020], dist_units='km').restrict_to_dates(["20200301","20200331"])

/home/mschade/anaconda3/envs/gapmapbox/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# f21_mar = gtfs_kit_scissors.cut_dates(gk.read_feed(paths[2021], dist_units='km'), ["20210301","20210331"], validate=False)
f21_mar = gk.read_feed(paths[2021], dist_units='km').restrict_to_dates(["20210301","20210331"])


## Olpe

Pick a Kreis with grave differences.

In [6]:
kreise_gdf = gpd.read_file("/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/geo/vg250-ew_12-31.utm32s.shape.ebenen/vg250-ew_ebenen_1231/VG250_KRS.shp"
                          ).to_crs("epsg:4326")

In [7]:
olpe_gdf = kreise_gdf[kreise_gdf.GEN=="Olpe"]

In [8]:
# f20_mar_olpe = gtfs_kit_scissors.cut_geom(f20_mar, olpe_gdf, validate=False)
f20_mar_olpe = f20_mar.restrict_to_area(olpe_gdf)

In [9]:
# f21_mar_olpe = gtfs_kit_scissors.cut_geom(f21_mar, olpe_gdf, validate=False)
f21_mar_olpe = f21_mar.restrict_to_area(olpe_gdf)

In [10]:
len(f20_mar_olpe.routes)

5

In [12]:
f20_mar_olpe.routes

,route_long_name,route_short_name,agency_id,route_type,route_id
6877,301,301,185,3,12502
10283,491,491,166,3,6890
21198,RB91,RB91,195,2,6024
21199,RB92,RB92,195,2,16859
21221,RE16,RE16,195,2,18298


In [13]:
len(f21_mar_olpe.routes)

49

In [15]:
f21_mar_olpe.routes

,route_long_name,route_short_name,agency_id,route_type,route_id
6341,301,301,14,3,11838
7119,335,335,226,3,12552
14336,70,70,162,3,19856
14806,72,72,162,3,16959
19578,A514,A514,218,3,4742
19579,A522,A522,218,3,2640
19615,ALF96,ALF96,162,3,17061
20955,L511,L511,218,3,2625
20956,L512,L512,218,3,3603
20957,L513,L513,218,3,13442


In [16]:
f21_mar_olpe.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
45,14,Verkehrsverbund Rhein-Sieg,http://www.bahn.de,Europe/Berlin,de
70,162,Märkische VG,http://www.bahn.de,Europe/Berlin,de
132,218,Verkehrsgemeinschaft Westfalen-Süd,http://www.bahn.de,Europe/Berlin,de
136,221,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
141,226,VGM/VRL,http://www.bahn.de,Europe/Berlin,de


In [17]:
f20_mar_olpe.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
74,166,WestVerkehr,http://www.west-verkehr.de,Europe/Berlin,de
95,185,Verkehrsverbund Rhein-Sieg,http://www.bahn.de,Europe/Berlin,de
106,195,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de


Es sieht so aus, als ob hier einfach ein Haufen Agenturen fehlten in 2020--wobei eine auch weggefallen ist...

Wie sieht es aus mit den Bussen, die es nicht gibt?

In [59]:
f20_mar.routes[f20_mar.routes.agency_id=='166']

,route_long_name,route_short_name,agency_id,route_type,route_id
8596,401,401,166,3,20193
8621,402,402,166,3,8558
8644,403,403,166,3,5365
8684,405,405,166,3,12450
8699,406,406,166,3,22537
8714,407,407,166,3,7568
8728,408,408,166,3,12490
8792,410,410,166,3,10210
8819,411,411,166,3,8817
8847,412,412,166,3,18070


## Die Schlimmsten...

### Anhalt-Bitterfeld

In [18]:
anh_bitt_gdf = kreise_gdf[kreise_gdf.GEN=="Anhalt-Bitterfeld"]

In [20]:
# f20_mar_olpe = gtfs_kit_scissors.cut_geom(f20_mar, olpe_gdf, validate=False)
f20_mar_ab = f20_mar.restrict_to_area(anh_bitt_gdf)

In [21]:
# f21_mar_olpe = gtfs_kit_scissors.cut_geom(f21_mar, olpe_gdf, validate=False)
f21_mar_ab = f21_mar.restrict_to_area(anh_bitt_gdf)

In [22]:
len(f20_mar_ab.routes)

75

In [28]:
f20_mar_ab.routes.agency_id.value_counts()

35     63
195    10
271     1
275     1
Name: agency_id, dtype: int64

In [24]:
len(f21_mar_ab.routes)

17

In [29]:
f21_mar_ab.routes.agency_id.value_counts()

221    8
126    6
79     1
156    1
242    1
Name: agency_id, dtype: int64

In [30]:
f20_mar_ab.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
106,195,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
191,271,NJL,http://www.bahn.de,Europe/Berlin,de
195,275,Omnibusbetrieb Saalekreis GmbH,http://www.bahn.de,Europe/Berlin,de
225,35,Vetter GmbH,http://www.bahn.de,Europe/Berlin,de


In [31]:
f21_mar_ab.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
30,126,Hallesche Verkehrs-AG,http://www.bahn.de,Europe/Berlin,de
63,156,Omnibusbetrieb Saalekreis GmbH,http://www.bahn.de,Europe/Berlin,de
136,221,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
159,242,Dessauer Verkehrsgesellschaft,http://www.bahn.de,Europe/Berlin,de
276,79,NJL,http://www.bahn.de,Europe/Berlin,de


### Berlin

In [42]:
berlin_gdf = kreise_gdf[kreise_gdf.GEN=="Berlin"]

In [43]:
f20_mar_berlin = f20_mar.restrict_to_area(berlin_gdf)

In [44]:
f21_mar_berlin = f21_mar.restrict_to_area(berlin_gdf)

In [45]:
len(f20_mar_berlin.routes)

316

In [46]:
f20_mar_berlin.routes

,route_long_name,route_short_name,agency_id,route_type,route_id
1495,100,100,157,3,22167
1546,101,101,157,3,12160
1696,104,104,157,3,2184
1782,106,106,157,3,17845
1821,107,107,157,3,1990
...,...,...,...,...,...
22455,X69,X69,157,3,13852
22457,X7,X7,157,3,6690
22462,X76,X76,157,3,296
22470,X83,X83,157,3,8491


In [47]:
len(f21_mar_berlin.routes)

342

In [48]:
f21_mar_berlin.routes

,route_long_name,route_short_name,agency_id,route_type,route_id
72,034,034,228,3,1177
77,039,039,228,3,14269
80,042,042,228,3,22747
106,065,065,228,3,21662
112,070,070,228,3,5019
...,...,...,...,...,...
23661,X69,X69,249,3,15781
23663,X7,X7,249,3,17680
23664,X71,X71,249,3,18870
23671,X76,X76,249,3,11820


In [40]:
f20_mar_berlin.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
0,1,mob,http://www.bahn.de,Europe/Berlin,de
64,157,BVG Berliner Verkehrsbetriebe,http://www.bahn.de,Europe/Berlin,de
91,181,Havelbus Verkehrsgesellschaft mb,http://www.bahn.de,Europe/Berlin,de
93,183,Verkehrsgesell. Teltow-Fläming,http://www.bahn.de,Europe/Berlin,de
96,186,Regiobus Potsdam Mittelmark GmbH,http://www.bahn.de,Europe/Berlin,de
106,195,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
123,21,Uckermärkische VG ab 12.12.2010,http://www.bahn.de,Europe/Berlin,de
135,220,Oberhavel Verkehrsgesellschaft,http://www.bahn.de,Europe/Berlin,de
171,253,Verkehrsbetrieb Potsdam GmbH,http://www.bahn.de,Europe/Berlin,de
205,284,VBB-Lieferungen Tabellenplaner,http://www.bahn.de,Europe/Berlin,de


In [41]:
f21_mar_berlin.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
44,139,S Bahn Berlin GmbH,http://www.bahn.de,Europe/Berlin,de
57,150,Regionalverkehr Spreewald,http://www.bahn.de,Europe/Berlin,de
60,153,Havelbus Verkehrsgesellschaft mb,http://www.bahn.de,Europe/Berlin,de
84,175,VBB-Lieferungen Tabellenplaner,http://www.bahn.de,Europe/Berlin,de
94,184,Verkehrsbetrieb Potsdam GmbH,http://www.bahn.de,Europe/Berlin,de
102,191,Verkehrsgesell. Teltow-Fläming,http://www.bahn.de,Europe/Berlin,de
109,198,mobus,http://www.bahn.de,Europe/Berlin,de
136,221,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
143,228,Flixbus,http://www.bahn.de,Europe/Berlin,de
151,235,Busverkehr Oder-Spree ab 12.2005,http://www.bahn.de,Europe/Berlin,de


This doesn't tell us much, yet. How about trip stats for first week in March?

In [56]:
firstweek21 = ["2021030"+str(n) for n in range(1,8)]
firstweek20 = ["2020030"+str(n) for n in range(2,9)]

In [57]:
gk.compute_feed_stats(f20_mar_berlin, gk.compute_trip_stats(f20_mar_berlin), firstweek20)

,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time,date
0,9590,314,42493,42493,40452,0.0,27009.311667,0.0,1692,16:50:00,16:50:12,20200302
0,9590,314,42498,42498,40457,0.0,27023.595000,0.0,1692,16:50:00,16:50:12,20200303
0,9590,314,42493,42493,40452,0.0,27016.061667,0.0,1691,16:50:00,16:50:12,20200304
0,9590,314,42496,42496,40457,0.0,27041.195000,0.0,1694,16:50:00,16:50:12,20200305
0,9590,315,41467,41467,39871,0.0,26331.266667,0.0,1675,07:30:00,07:30:12,20200306
0,9590,282,18613,18613,17110,0.0,12155.265000,0.0,772,20:38:00,20:38:06,20200307
0,9590,286,28192,28192,26151,0.0,17761.186667,0.0,1105,17:26:30,17:27:00,20200308


In [53]:
gk.compute_feed_stats(f21_mar_berlin, gk.compute_trip_stats(f21_mar_berlin), firstweek21)

,num_stops,num_routes,num_trips,num_trip_starts,num_trip_ends,service_distance,service_duration,service_speed,peak_num_trips,peak_start_time,peak_end_time,date
0,17958,307,42375,42375,40309,7.936254e+08,27454.736667,28906.683958,1705,16:12:30,16:12:54,20210301
0,17958,307,42358,42358,40296,7.839519e+08,27356.636667,28656.736422,1703,16:12:30,16:12:54,20210302
0,17958,307,42357,42357,40294,7.845121e+08,27360.386667,28673.280979,1703,16:12:30,16:12:54,20210303
0,17958,307,42354,42354,40292,7.840311e+08,27357.503333,28658.724052,1703,16:12:30,16:12:54,20210304
0,17958,307,41371,41371,39781,7.545051e+08,26611.290000,28352.819696,1693,07:26:30,07:26:54,20210305
0,17958,257,11204,11204,9717,3.296547e+08,7987.963333,41268.924433,653,21:37:24,21:37:30,20210306
0,17958,254,8560,8560,6668,2.831509e+08,6350.683333,44585.901118,621,21:56:30,21:57:00,20210307


In [58]:
gk.compute_stop_stats(f20_mar_berlin, firstweek20)

,stop_id,num_routes,num_trips,start_time,end_time,max_headway,min_headway,mean_headway,date
0,100040,1.0,12.0,03:53:00,27:23:00,NaN,NaN,NaN,20200302
1,100057,2.0,125.0,03:12:00,26:42:00,10.0,6.0,8.765432,20200302
2,100114,1.0,228.0,04:14:00,24:44:00,5.0,4.0,4.612903,20200302
3,100116,2.0,277.0,03:44:00,27:14:00,6.0,3.0,3.789474,20200302
4,1002,5.0,185.0,03:05:00,27:04:00,10.0,3.0,6.663551,20200302
...,...,...,...,...,...,...,...,...,...
8851,9973,1.0,6.0,10:03:00,20:03:00,120.0,120.0,120.000000,20200308
8852,99752,1.0,98.0,03:31:00,26:59:00,20.0,10.0,11.387097,20200308
8853,99939,1.0,152.0,03:37:00,27:07:00,14.0,6.0,7.489362,20200308
8854,99984,1.0,3.0,08:23:00,16:23:00,240.0,240.0,240.000000,20200308


In [59]:
gk.compute_stop_stats(f21_mar_berlin, firstweek21)

,stop_id,num_routes,num_trips,start_time,end_time,max_headway,min_headway,mean_headway,date
0,100082,2.0,90.0,03:30:00,27:00:00,21.0,10.0,12.482143,20210301
1,100107,1.0,46.0,05:22:00,20:22:00,21.0,19.0,20.000000,20210301
2,100189,1.0,217.0,04:13:00,24:53:00,5.0,5.0,5.000000,20210301
3,100214,2.0,149.0,04:47:00,24:12:00,11.0,1.0,6.472727,20210301
4,100347,1.0,9.0,03:43:00,27:13:00,NaN,NaN,NaN,20210301
...,...,...,...,...,...,...,...,...,...
7852,99430,1.0,12.0,20:14:00,23:54:00,NaN,NaN,NaN,20210307
7853,9944,1.0,11.0,20:47:00,24:07:00,NaN,NaN,NaN,20210307
7854,99562,1.0,12.0,20:34:00,24:14:00,NaN,NaN,NaN,20210307
7855,99612,1.0,6.0,25:35:00,28:05:00,NaN,NaN,NaN,20210307


## Wittenberg

Schauen wir uns mal einen weiteren Kreis an. Wittenberg.

In [27]:
wittenb_gdf = kreise_gdf[kreise_gdf.GEN=="Wittenberg"]
wittenb_gdf

,ADE,GF,BSG,ARS,AGS,SDV_ARS,GEN,BEZ,IBZ,BEM,...,ARS_0,AGS_0,WSK,EWZ,KFL,DEBKG_ID,RS,SDV_RS,RS_0,geometry
377,4,4,1,15091,15091,150910375375,Wittenberg,Landkreis,43,--,...,150910000000,15091000,2009-01-01,124953,1930.78,DEBKGDL20000E14U,15091,150910375375,150910000000,"POLYGON ((12.37756 52.04493, 12.38543 52.04359..."


In [28]:
f20_mar_wittenb = gtfs_kit_scissors.cut_geom(f20_mar, wittenb_gdf, validate=False)

Stops reduction: 99.7%
Stop_times reduction: 99.8%
Trips reduction: 99.7%
Calendar reduction: 98.8%
Calendar_dates reduction: 99.7%
Routes reduction: 99.7%
Agencies reduction: 98.2%
Transfers reduction: 100.0%


In [29]:
f21_mar_wittenb = gtfs_kit_scissors.cut_geom(f21_mar, wittenb_gdf, validate=False)

Stops reduction: 100.0%
Stop_times reduction: 100.0%
Trips reduction: 99.9%
Calendar reduction: 99.6%
Calendar_dates reduction: 99.0%
Routes reduction: 99.9%
Agencies reduction: 98.5%


In [30]:
f20_mar_wittenb.routes

,route_long_name,route_short_name,agency_id,route_type,route_id
5364,236,236,35,3,1659
5413,238,238,35,3,16882
6865,300,300,35,3,9168
6890,301,301,35,3,14404
6917,302,302,35,3,6965
6947,303H,303H,35,3,15184
6948,303W,303W,35,3,13100
6962,304,304,35,3,19720
7001,306,306,35,3,3958
7022,307,307,35,3,515


In [31]:
f21_mar_wittenb.routes

,route_long_name,route_short_name,agency_id,route_type,route_id
4133,22,22,242,3,12086
11132,542,542,194,3,20786
15483,751,751,27,3,6873
20101,DPN,DPN,221,2,20259
20106,DWE,DWE,242,2,19489
20861,Intercity,IC,221,2,13131
20862,Intercity-Express,ICE,221,2,13923
22261,RB51,RB51,221,2,8084
22344,RE14,RE14,221,2,14431
22369,RE3,RE3,221,2,8186


In [32]:
f20_mar_wittenb.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
79,170,Verkehrsmanagement Elbe-Elster,http://www.bahn.de,Europe/Berlin,de
106,195,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
225,35,Vetter GmbH,http://www.bahn.de,Europe/Berlin,de
276,81,OVH Heideland,http://www.bahn.de,Europe/Berlin,de


In [33]:
f21_mar_wittenb.agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang
105,194,Verkehrsmanagement Elbe-Elster,http://www.bahn.de,Europe/Berlin,de
136,221,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
159,242,Dessauer Verkehrsgesellschaft,http://www.bahn.de,Europe/Berlin,de
189,27,OVH Heideland,http://www.bahn.de,Europe/Berlin,de


Hier scheint die "Vetter GmbH" einfach weggefallen zu sein... Die sind wohl schon länger umstritten: https://www.wittenbergersonntag.de/artikel/826

## Leer

In [36]:
keyword = "Leer"
from IPython.display import display
leer_gdf = kreise_gdf[kreise_gdf.GEN==keyword]

f20_mar_leer = gtfs_kit_scissors.cut_geom(f20_mar, leer_gdf, validate=False)

f21_mar_leer = gtfs_kit_scissors.cut_geom(f21_mar, leer_gdf, validate=False)

display(f20_mar_leer.routes)

display(f21_mar_leer.routes)

display(f20_mar_leer.agency)

display(f21_mar_leer.agency)

Stops reduction: 99.6%
Stop_times reduction: 99.9%
Trips reduction: 99.9%
Calendar reduction: 98.5%
Calendar_dates reduction: 85.3%
Routes reduction: 99.6%
Agencies reduction: 97.7%
Transfers reduction: 100.0%
Stops reduction: 99.7%
Stop_times reduction: 99.9%
Trips reduction: 99.9%
Calendar reduction: 99.5%
Calendar_dates reduction: 99.7%
Routes reduction: 99.7%
Agencies reduction: 98.1%


,route_long_name,route_short_name,agency_id,route_type,route_id
7806,35,RE,195,2,10929
7885,354,354,74,3,1600
9785,460,460,116,3,14989
9942,467,467,116,3,20826
10002,470,470,116,3,418
...,...,...,...,...,...
20436,N63,N63,116,3,3850
21209,RE1,RE1,195,2,7016
21219,RE15,RE15,195,2,2430
21852,SCH,SCH,195,4,17976


,route_long_name,route_short_name,agency_id,route_type,route_id
7453,354,354,268,3,13432
9497,460,460,161,3,12592
9669,467,467,161,3,118
9734,470,470,161,3,7027
9830,476,476,161,3,23003
...,...,...,...,...,...
22335,RE1,RE1,221,2,11786
22346,RE15,RE15,221,2,7981
22377,RE35,RE35,221,2,21122
22405,RE56,RE56,221,2,8999


,agency_id,agency_name,agency_url,agency_timezone,agency_lang
19,116,WEB Weser-Ems-Bus,https://www.weser-ems-bus.de/weseremsbus,Europe/Berlin,de
47,141,Andressen Reisen,http://www.bahn.de,Europe/Berlin,de
106,195,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
208,287,WMS LK Emsland,http://www.bahn.de,Europe/Berlin,de
268,74,GER Gerdes Reisen,http://www.bahn.de,Europe/Berlin,de


,agency_id,agency_name,agency_url,agency_timezone,agency_lang
49,143,Andressen Reisen,http://www.bahn.de,Europe/Berlin,de
69,161,WEB Weser-Ems-Bus,https://www.weser-ems-bus.de/weseremsbus,Europe/Berlin,de
116,203,WMS LK Emsland,http://www.bahn.de,Europe/Berlin,de
136,221,Gesamtnetz der deutschen Bahn,http://www.bahn.de,Europe/Berlin,de
187,268,GER Gerdes Reisen,http://www.bahn.de,Europe/Berlin,de


In [38]:
f20_mar_leer.stops

,stop_name,parent_station,stop_code,zone_id,stop_id,stop_lat,stop_lon,location_type,wheelchair_boarding,platform_code
1520,"Aaltuikerei, Abzw. Pogum, Mast 1",631728,RATA1,NaN,113622,53.280727,7.266798,NaN,NaN,NaN
1521,"Aaltuikerei, Abzw. Pogum, Mast 2",631728,RATA2,NaN,564146,53.280710,7.266768,NaN,NaN,NaN
1522,"Aaltuikerei, Siedlung, Mast 1",616651,9385201,NaN,567367,53.277260,7.265648,NaN,NaN,NaN
1523,"Aaltuikerei, Siedlung, Mast 2",616651,9385202,NaN,635725,53.277260,7.265648,NaN,NaN,NaN
17525,"Altschwoog, B70/Zu den Eichen Altschwoog",11420,NaN,NaN,430493,53.280350,7.455760,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
705225,Wymeer Molkerei,NaN,NaN,NaN,252793,53.139465,7.253386,1.0,NaN,NaN
705226,Wymeer Müller,NaN,NaN,NaN,73792,53.134598,7.209919,1.0,NaN,NaN
705227,Wymeer Polizei,NaN,NaN,NaN,5186,53.134890,7.219306,1.0,NaN,NaN
705228,Wymeer Schule,NaN,NaN,NaN,422860,53.135216,7.224777,1.0,NaN,NaN


In [39]:
f21_mar_leer.stops

,stop_name,parent_station,stop_code,stop_id,stop_lat,stop_lon,location_type
17864,Altebeek 153 Jheringsfehn,116548,NaN,812435,53.303640,7.533470,NaN
17865,Altebeek 153 Jheringsfehn,116548,NaN,925402,53.303650,7.533680,NaN
22864,"Altschwoog, B70/Zu den Eichen, Mast 1",865260,NaN,132420,53.280550,7.455984,NaN
22865,"Altschwoog, B70/Zu den Eichen, Mast 2",865260,NaN,682440,53.279987,7.455705,NaN
26995,"Ammersum, Dieken, Mast 1",175731,NaN,158308,53.240276,7.663020,NaN
...,...,...,...,...,...,...,...
931673,Wymeer Ost-West-Straße/Moorstraße,NaN,NaN,135175,53.122555,7.210494,1.0
931674,Wymeer Wymeerster Hauptstraße 61,NaN,NaN,598833,53.134890,7.219306,1.0
931675,Wymeer Wymeerster Hauptstraße/Karkpad,NaN,NaN,718837,53.135372,7.240371,1.0
931676,Wymeer Wymeerster Hauptstraße/Molkerei,NaN,NaN,101825,53.139465,7.253386,1.0


In [43]:
f21_mar_leer.calendar

,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,service_id
201,0,0,0,0,0,0,1,20201213,20210321,14949
220,0,0,0,0,0,0,1,20201213,20210321,29442
288,0,0,0,0,0,0,1,20201213,20210321,72538
292,0,0,0,0,0,0,1,20201213,20210321,74914
399,0,0,0,0,0,0,1,20201220,20210321,41097
...,...,...,...,...,...,...,...,...,...,...
4102,0,0,0,0,0,0,1,20210328,20210328,82593
4129,0,0,0,0,1,0,0,20210326,20210326,83036
4378,0,0,0,0,1,0,0,20210326,20210326,87841
4618,0,0,0,0,1,0,0,20210326,20210326,91762


In [44]:
f20_mar_leer.calendar

,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,service_id
177,0,0,0,0,0,0,1,20191215,20200315,18119
180,0,0,0,0,0,0,1,20191215,20200315,2343
198,0,0,0,0,0,0,1,20191215,20200322,8771
255,0,0,0,0,0,0,1,20191215,20200329,925
272,0,0,0,0,0,0,1,20191215,20200405,22635
...,...,...,...,...,...,...,...,...,...,...
688,0,0,0,0,0,1,0,20200328,20200328,7411
736,0,0,0,0,1,0,0,20200306,20200306,8402
756,0,0,0,0,0,1,0,20200328,20200328,9075
771,1,0,0,0,0,0,0,20200302,20200302,9674


In [166]:
def uniqueGdf(gdf):
    uni_gdf = gdf.iloc[[0]]
    for i, r in gdf.iterrows():
        if not (any([p.equals(r.geometry) for p in uni_gdf.geometry])):
            uni_gdf = uni_gdf.append(r) 
    return uni_gdf

In [203]:
point_counts_21 = gpd.sjoin(uni_gdf[["stop_id","geometry"]], stops_harz_21, op='intersects'
         ).groupby("stop_id_left"
                  ).agg({"stop_id_right":"count",
                         "geometry":"first", 
                         "stop_name":"first"
                        })

In [204]:
point_counts_21

,stop_id_right,geometry,stop_name
stop_id_left,,,
100636,2,POINT (11.11824 51.93056),Groß Quenstedt Heerstr.
101037,2,POINT (11.25363 51.85048),Hedersleben Neubauten L66
101161,2,POINT (10.84136 51.91468),Mulmke Kreisstr.
102493,2,POINT (10.82037 51.97674),Dardesheim Alter Bahnhof
102603,1,POINT (11.00846 51.90848),Kosakengrab
...,...,...,...
98858,1,POINT (10.72649 51.77040),Drei Annen Hohne Bf.
98910,3,POINT (11.04097 51.75330),Thale Karl-Marx-Str.
991,1,POINT (11.02359 51.91115),Sargstedter Siedlung


In [205]:
display(point_counts_21.stop_id_right.value_counts())
print(point_counts_21.stop_id_right.sum())

2    1094
1     496
3     333
4       3
Name: stop_id_right, dtype: int64

3695


In [206]:
point_counts_21[point_counts_21.stop_id_right>2]

,stop_id_right,geometry,stop_name
stop_id_left,,,
103329,3,POINT (11.33130 51.68451),Wieserode
105896,3,POINT (10.73903 51.78809),Wernigerode Drt_Drw 008 P8
109894,3,POINT (11.11911 51.60931),Neudorf Kirche
116481,3,POINT (10.86625 52.01599),Westerburg Burg
116718,3,POINT (11.05044 51.74773),Thale H.-Heine Str.
...,...,...,...
943229,3,POINT (11.13922 51.64013),Harzgerode Parkplatz
944087,3,POINT (10.90389 51.86727),Derenburg Gewerbegebiet
946055,3,POINT (10.69046 51.86128),Ilsenburg Sekundarschule


In [202]:
stops_harz_21[stops_harz_21.stop_id.isin(point_counts_21[point_counts_21.stop_id_right>2].index)]

,stop_name,parent_station,stop_code,stop_id,stop_lat,stop_lon,location_type,geometry,ADE,GF,...,NUTS,ARS_0,AGS_0,WSK,EWZ,KFL,DEBKG_ID,RS,SDV_RS,RS_0
2407,Abbenrode Wendeplatz,NaN,NaN,199447,51.929550,10.626381,1.0,POINT (10.62638 51.92955),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000
15102,Allrode Rosenweg,NaN,NaN,870686,51.674480,10.974545,1.0,POINT (10.97455 51.67448),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000
25278,Am Silbernen Mann Drängetal Gleis 1,NaN,NaN,13286,51.798153,10.729809,1.0,POINT (10.72981 51.79815),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000
25281,Am Silbernen Mann Drängetal Gleis 2,NaN,NaN,536534,51.798153,10.729896,1.0,POINT (10.72990 51.79815),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000
56925,Bad Suderode Markt,NaN,NaN,670974,51.728565,11.120022,1.0,POINT (11.12002 51.72857),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
907213,Wienrode Harzstr.,NaN,NaN,732770,51.765965,10.972901,1.0,POINT (10.97290 51.76597),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000
907221,Wienrode Mühlenstr.,NaN,NaN,394285,51.766594,10.973929,1.0,POINT (10.97393 51.76659),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000
908954,Wieserode,NaN,NaN,103329,51.684513,11.331302,1.0,POINT (11.33130 51.68451),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000
933287,Wülperode Zum Dören,NaN,NaN,491082,51.981240,10.601442,1.0,POINT (10.60144 51.98124),4,4,...,DEE09,150850000000,15085000,2009-01-01,213310,2104.77,DEBKGDL20000E6Y3,15085,150850135135,150850000000


In [192]:
point_counts_20 = gpd.sjoin(uniqueGdf(stops_harz_20)[["stop_id","geometry"]],
                            stops_harz_20, 
                            op='intersects'
         ).groupby("stop_id_left"
                  ).agg({"stop_id_right":"count",
                         "geometry":"first"
                        })

KeyboardInterrupt: 

In [207]:
display(point_counts_20.stop_id_right.value_counts())
print(point_counts_20.stop_id_right.sum())

1    1449
2     407
3       1
Name: stop_id_right, dtype: int64

2266


In [ ]:
point